# Anotações

- dado uma sequencia de entrada eu quero gerar uma sequencia de saida
    - nesse caso da gente vamos prever uma sequencia de tamanho igual a 10, quanto maior quanto mais preciso é o modelo
    - não tem nada infinito! sempre tem uma limitação
        - nós não temos limite de saida de texto

- O modelo trnasfomer tem algumas partes principais:
    - Camada
        - Embedding -> transforma tokens em vetor numérico de tamanho fixo
        - Mecanismo de atenção -> Permite que modelo foque em diferentes partes da entrada
        - Encoder e Decoder -> processam os dados sequencialmente
        - Linear e softmax -> para predições finais

- nesse curso vamos implementar o mvp, sem ser com o encoder e decoder, eles melhoram a performance.

# Imports

In [2]:
import numpy as np

# Hiperparametros

In [3]:
# dimensão do modelo
dim_model = 64

# comprimento da sequencia
seq_length = 10

# tamanho do vocabulário(ele que vai permitir o contexto)
vocab_size = 100

# Camada de embedding

## A função da camanda de embedding é converter entradas sequenciais em vetores densos de tamnho  fixo, esses vetores são conhecidos como embedding
  - são essenciais pelo motivo de que eles fazem as proximas etapas possíveis e mantendo a sequencia e contexto

In [4]:
def embedding(input, vocab_size, dim_model):
    embed = np.random.randn(vocab_size, dim_model)

    return np.array([embed[i] for i in input])

# Mecanismo de atenção

## Componentes q, k v


In [7]:
#  - Tem tipos de atenção vamos ver a do tipo scaled dot-product, que utiliza 3 componentes principais -> q, k e v
#   - Q
#     - parte que estamos interessado no momento(trecho da frase que estamos tentando trduzir por exemplo)
#     - em um modelo trnaformer tmeos qeu tem um Q associado a cada posição da sequencia de entrada/saida
#       - essas queries são usadas para determinar quao importante é cada parte da entrada para essa posição particular
#   - K
#     - as keys são usadas para pontuar cada parte da entrada tambvem e elas são comparadas com as queries para determinar graud e atenção que cad aparte deve receber
#     - elas são operadas para indicar quão relevnatre cada parte da entrada é apra parte representada pela query
#   - V
#     - os valores tem a informação rela que queremos na saida
#     - depois que o q e o k indicam onde focar o v é usado para compor a dsaida do mecanismo de atenção
#     - cada vaelue é associado a uma key e ele é o resultado a partir de uma ponderação de uma operação entre as queries e keys
#     - o scaled dot product vem do calculo do conjunto de pontuações calculado por meio de um produto escalar entre q e k

## Softmax

In [8]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=-1).reshape(-1, 1)

## Scale dot product

- De forma simples estamos dando peso para nossas combinações dentro da embedding
  - esses numeros representam o relacionamento basico entre as sequencias com base no vocabulario, isso que o modelo aprende

- como é ajustado todos esses pesos durante o treinamento?
  - da embedding e q,k e v

In [18]:
def scale_dot_product_attention(Q,K,V):
    matmul_qk = np.dot(Q, K.T)

    depth = K.shape[-1]

    # resultado da matriz
    logits = matmul_qk / np.sqrt(depth)

    attention_weights = softmax(logits)

    output = np.dot(attention_weights, V)

    return output


## Softmax e linear

In [10]:
def linear_and_softmax(input):
    weights = np.random.randn(dim_model, vocab_size)

    logits = np.dot(input, weights)

    return softmax(logits)

# Final model

In [12]:
def transformer_model(input):
    embedding_input = embedding(input, vocab_size, dim_model)

    attention_output = scale_dot_product_attention(embedding_input, embedding_input, embedding_input)

    ouput_probabilities = linear_and_softmax(attention_output)

    ouput_indices = np.argmax(ouput_probabilities, axis=1)

    return ouput_indices

# Usando modelo para as previsões

In [13]:
input_sequence = np.random.randint(0, vocab_size, seq_length)

In [14]:
print(f'seq de entrada: {input_sequence}')

seq de entrada: [58 35 77 51 94 42 37 79 77 13]


In [19]:
output_sequence = transformer_model(input_sequence)

In [20]:
print(f'saida do modelo: {output_sequence}')

saida do modelo: [23 61 98 97 38 10 43 92 98 68]
